In [ ]:
import pandas as pd
from shapely.geometry import Polygon

def calculate_polygon_area(polygon):
    """Calculate the area of a polygon"""
    return polygon.area

def calculate_polygon_intersection_area(poly1, poly2):
    """Calculate the intersection area of two polygons"""
    intersection = poly1.intersection(poly2)
    return intersection.area

def fix_polygon_self_intersection(polygon):
    """Correct polygon self intersection error"""
    return polygon.buffer(0)

def create_polygon(points):
    """Create polygons based on points and correct self intersection errors"""
    if len(points) > 2:
        polygon = Polygon(points).convex_hull 
        return fix_polygon_self_intersection(polygon)
    else:
        return None

def process_files(excel_file1, excel_file2, output_file):
    df1 = pd.read_excel(excel_file1)
    df2 = pd.read_excel(excel_file2)

    # Record the boxes that need to be deleted
    to_delete1 = []
    to_delete2 = []

    # Group by image
    grouped_by_filename1 = df1.groupby('Prediction Filename')
    grouped_by_filename2 = df2.groupby('Prediction Filename')

    all_filenames = set(df1['Prediction Filename']).union(set(df2['Prediction Filename']))

    for filename in all_filenames:
        # Obtain box data for each image
        group1 = grouped_by_filename1.get_group(filename) if filename in grouped_by_filename1.groups else pd.DataFrame()
        group2 = grouped_by_filename2.get_group(filename) if filename in grouped_by_filename2.groups else pd.DataFrame()

        # create Polygon
        box_polygons1 = {
            box_id: (create_polygon(box_group[['X', 'Y']].values.tolist()), box_group.iloc[0]['Confidence'])
            for box_id, box_group in group1.groupby('Box ID')
            if create_polygon(box_group[['X', 'Y']].values.tolist()) is not None
        }
        box_polygons2 = {
            box_id: (create_polygon(box_group[['X', 'Y']].values.tolist()), box_group.iloc[0]['Confidence'])
            for box_id, box_group in group2.groupby('Box ID')
            if create_polygon(box_group[['X', 'Y']].values.tolist()) is not None
        }

        # Compare and preserve boxes
        for box_id1, (poly1, conf1) in box_polygons1.items():
            for box_id2, (poly2, conf2) in box_polygons2.items():
                intersection_area = calculate_polygon_intersection_area(poly1, poly2)
                poly1_area = calculate_polygon_area(poly1)
                poly2_area = calculate_polygon_area(poly2)

                if intersection_area / poly1_area > 0.8 and intersection_area / poly2_area > 0.8:
                    # Box 1 and Box 2 both occupy 80% of the intersection, retaining the box with higher reliability and eliminating the other box
                    if conf1 >= conf2:
                        to_delete2.append((filename, box_id2))
                    else:
                        to_delete1.append((filename, box_id1))
                elif intersection_area / poly1_area > 0.8 and intersection_area / poly2_area < 0.5:
                    to_delete1.append((filename, box_id1))
                elif intersection_area / poly2_area > 0.8 and intersection_area / poly1_area < 0.5:
                    to_delete2.append((filename, box_id2))

    #Delete the box that needs to be deleted
    for filename, box_id in to_delete1:
        df1 = df1[~((df1['Prediction Filename'] == filename) & (df1['Box ID'] == box_id))]
    for filename, box_id in to_delete2:
        df2 = df2[~((df2['Prediction Filename'] == filename) & (df2['Box ID'] == box_id))]

    result_df = pd.concat([df1, df2], ignore_index=True)

    result_df.to_excel(output_file, index=False)

process_files('RGB.xlsx', 'red_green_nir.xlsx', 'filtered_results.xlsx')


In [7]:
import os
import pandas as pd

def replace_image_path(df, old_path, new_path):
    df['Image Path'] = df['Image Path'].str.replace(old_path, new_path)
    return df

def process_excel_files(folder_path, old_path, new_path):
    df = pd.read_excel(folder_path)
    df = replace_image_path(df, old_path, new_path)
    df.to_excel(folder_path, index=False)
    print(f"Processed {folder_path}") 

base_folder = 'filtered_results.xlsx'  
new_path = 'red_green_nir' # Path that needs to be modified
old_path = 'RGB' # Path that needs to be modified

process_excel_files(base_folder, old_path, new_path)

Processed filtered_results.xlsx


In [4]:
import pandas as pd
import cv2
import os
from PIL import Image
import numpy as np

color_map = {
    0: (255, 0, 0),  # blue
    1: (0, 0, 255)   # red
}

excel_file = 'filtered_results.xlsx'  
df = pd.read_excel(excel_file)

# Get a unique list of image paths
image_paths = df['Image Path'].unique()

for image_path in image_paths:
    try:
        # Using PIL to read images
        with Image.open(image_path) as pil_image:
            image = np.array(pil_image)
            # If the image is a grayscale image, it needs to be converted to RGB
            if len(image.shape) == 2:
                image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
            elif image.shape[2] == 4:  
                image = cv2.cvtColor(image, cv2.COLOR_RGBA2BGR)
            else:  # Ensure to convert RGB to BGR
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    except Exception as e:
        print(f"Error reading image: {image_path} with error: {e}")
        continue
    
    image_data = df[df['Image Path'] == image_path]
    
    grouped = image_data.groupby('Box ID')
    
    for box_id, group in grouped:
        # Obtain the classification and confidence level of the box
        class_id = group['Class'].iloc[0]
        confidence = group['Confidence'].iloc[0]
        color = color_map[class_id]
        
        points = group[['X', 'Y']].values
        
        # Draw a polygonal box
        points = points.reshape((-1, 1, 2))
        cv2.polylines(image, [points], isClosed=True, color=color, thickness=2)
        
        # Get the first point of the box as the position of the confidence text
        x, y = points[0][0]
        cv2.putText(image, f'{confidence:.2f}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)
    
    folder_name = image_data['Folder'].iloc[0]
    
    # Generate save path, add the original file name to the first column value
    output_filename = f'{os.path.basename(image_path)}'
    output_path = os.path.join('Save the folder path of the processed images in the box', output_filename)  
    
    # Create output folder (if it does not exist)
    os.makedirs('Save the folder path of the processed images in the box', exist_ok=True)
    
    if image is None or not isinstance(image, np.ndarray):
        print(f"Image data is invalid for: {image_path}")
        continue
    
    if not cv2.imwrite(output_path, image):
        print(f"Failed to save image: {output_path}")
    else:
        print(f'Saved: {output_path}')


Saved: images1\117.tiff
Saved: images1\29.tiff
Saved: images1\6.tiff
Saved: images1\114.tiff
Saved: images1\56.tiff
Saved: images1\8.tiff
Saved: images1\62.tiff
Saved: images1\110.tiff
Saved: images1\39.tiff
Saved: images1\81.tiff
Saved: images1\67.tiff
Saved: images1\113.tiff
Saved: images1\28.tiff
Saved: images1\18.tiff
Saved: images1\7.tiff
Saved: images1\11.tiff
Saved: images1\22.tiff
Saved: images1\46.tiff
Saved: images1\69.tiff
Saved: images1\98.tiff
Saved: images1\4.tiff
Saved: images1\59.tiff
Saved: images1\49.tiff
Saved: images1\40.tiff
Saved: images1\42.tiff
Saved: images1\91.tiff
Saved: images1\97.tiff
Saved: images1\116.tiff
Saved: images1\52.tiff
Saved: images1\105.tiff


In [8]:
import pandas as pd
import cv2
import os
from PIL import Image
import numpy as np

excel_file = 'filtered_results.xlsx' 
df = pd.read_excel(excel_file)

# Get a unique list of image paths
image_paths = df['Image Path'].unique()

for image_path in image_paths:
    try:
        with Image.open(image_path) as pil_image:
            image = np.array(pil_image)
            if len(image.shape) == 2:
                image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
            elif image.shape[2] == 4:  
                image = cv2.cvtColor(image, cv2.COLOR_RGBA2BGR)
            else:  
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    except Exception as e:
        print(f"Error reading image: {image_path} with error: {e}")
        continue
    
    mask = np.ones_like(image) * 255

    image_data = df[df['Image Path'] == image_path]
    
    grouped = image_data.groupby('Box ID')
    
    for box_id, group in grouped:
        points = group[['X', 'Y']].values
        
        points = points.reshape((-1, 1, 2))
        cv2.fillPoly(mask, [points], color=(0, 0, 0))
    
    # Set the part outside the frame to white
    image = cv2.bitwise_or(image, mask)
    
    folder_name = image_data['Folder'].iloc[0]
    
    output_filename = f'{os.path.basename(image_path)}'
    output_path = os.path.join('The folder path for saving segmented images', output_filename)  
    os.makedirs('The folder path for saving segmented images', exist_ok=True)
    
    if image is None or not isinstance(image, np.ndarray):
        print(f"Image data is invalid for: {image_path}")
        continue
    
    # 保存处理后的图片
    if not cv2.imwrite(output_path, image):
        print(f"Failed to save image: {output_path}")
    else:
        print(f'Saved: {output_path}')


Saved: images4\117.tiff
Saved: images4\29.tiff
Saved: images4\6.tiff
Saved: images4\114.tiff
Saved: images4\56.tiff
Saved: images4\8.tiff
Saved: images4\62.tiff
Saved: images4\110.tiff
Saved: images4\39.tiff
Saved: images4\81.tiff
Saved: images4\67.tiff
Saved: images4\113.tiff
Saved: images4\28.tiff
Saved: images4\18.tiff
Saved: images4\7.tiff
Saved: images4\11.tiff
Saved: images4\22.tiff
Saved: images4\46.tiff
Saved: images4\69.tiff
Saved: images4\98.tiff
Saved: images4\4.tiff
Saved: images4\59.tiff
Saved: images4\49.tiff
Saved: images4\40.tiff
Saved: images4\42.tiff
Saved: images4\91.tiff
Saved: images4\97.tiff
Saved: images4\116.tiff
Saved: images4\52.tiff
Saved: images4\105.tiff
